In [1]:
from collections import ChainMap
from datetime import date
import geoip2.database
import os
import pandas as pd
import us.states

Load latest city-by-IP database from [MaxMind](https://dev.maxmind.com/geoip/geoip2/geolite2/)

In [2]:
georeader = geoip2.database.Reader(
    os.path.join(
        [d for d in os.listdir() if d.startswith("GeoLite2-City")][-1],
        'GeoLite2-City.mmdb'
    )
)

In [3]:
def dir_last_month(present_date):
    """
    Paramater
    ---------
    present_date: date
        date to find directory for
        
    Returns
    path: str
        path of data directory for previous month
    """
    return(os.path.abspath(
        "-".join([
            str(present_date.year),
            str(present_date.month - 1)
        ])
    ) if present_date.month > 1 else os.path.abspath(
        "-".join([
            str(present_date.year - 1),
            str(12)
        ])
    ))


def geo_count(df, level=None):
    """
    Parameter
    ---------
    df: DataFrame
    
    level: string
        'city' or 'country'
    
    Returns
    -------
    count: int
    """
    return(len(geo_frame(df, level)))

    
def geo_frame(df, level=None):
    """
    Parameter
    ---------
    df: DataFrame
    
    level: string
        'city' or 'country'
    
    Returns
    -------
    df: DataFrame
    """
    if not level or level not in ["city", "country"]:
        print("geo_count: Specify\n\tlevel=\"city\" | \"country\"\nto get a count.")
        return(0)
    if level == "country":
        return(
            df["Country"].loc[df["Country"] != ""].drop_duplicates(
                "first"
            ).sort_values()
        )
    if level == "city":
        city_dupes = df.loc[
            (df.duplicated(["City", "Country"], keep=False))
            &
            (df["State or Province"] == "")
        ]
        return(
            df.loc[
                (df["City"] != "")
                &
                ~(
                    (df["City"].isin(city_dupes["City"]))
                    &
                    (df["State or Province"] == "")
                    &
                    (df["Country"].isin(city_dupes["Country"]))
                )
            ].drop_duplicates(
                ["City", "Country"]
            ).sort_values(
                ["Country", "City", "State or Province"]
            )
        )

    
def geo_from_ip(ip):
    """
    Parameter
    ---------
    ip : string
        IP address
        
    Returns
    -------
    city : string
    
    subdivision : string
    
    country : string
    """
    try:
        country = georeader.city(ip).country.names['en']
    except:
        country = ""
    try:
        subdivision = georeader.city(ip).subdivisions[0].names['en']
        subdivision = subdivision
    except:
        subdivision = country
    try:
        city = georeader.city(ip).city.names['en']
    except:
        city = subdivision
    return(city, subdivision, country)


def locsplit(loc):
    """
    Parameter
    ---------
    loc : string
       "City [, State|, Province|None], Country"
       
    Returns
    -------
    city : string
    
    subdivision : string
    
    country : string
    """
    loc_tup = tuple(loc.split(", "))
    if(len(loc_tup) == 2):
        return((loc_tup[0], "", loc_tup[1]))
    else:
        try:
            return((loc_tup[0], us.states.lookup(loc_tup[1]).name, loc_tup[2]))
        except:
            return(loc_tup)
        

def max_now_then(col, now, then):
    """
    Function to collect numbers max(this month, last month) to compensate for political changes
    (eg, renamed cities or countries)
    
    Parameters
    ----------
    col: str
        "CMI Total" or subset
        
    now: DataFrame
        appropriate DataFrame for this month's data
        
    then: DataFrame
        appropriate DataFrame for last month's data
        
    Returns
    -------
    maxes: dictionary
        one-item dictionary
        key: str
            col
        value: list
            2 item list, both ints
    """
    return({
        col:[
            max(
                then.loc[
                    "No. of Cities accessed data",
                    col
                ],
                geo_count(
                    now,
                    "city"
                )
            ),
            max(
                then.loc[
                    "No. of Countries accessed data",
                    col
                ],
                geo_count(
                    now,
                    "country"
                )
            )
        ]
    })
        
        
def print_geo_frame(df):
    """
    Parameter
    ---------
    df: DataFrame
       
    Returns
    -------
    geo_string : string
    """
    geo_string = ""
    for row in df.iterrows():
        if row[1]["City"] == row[1]["Country"] and row[1]["City"] == row[1]["State or Province"]:
            geo_string = "\n".join([
                geo_string,
                row[1]["Country"]
            ])
        elif row[1]["State or Province"] == "":
            geo_string = "\n".join([
                geo_string,
                ", ".join([
                    row[1]["City"],
                    row[1]["Country"]
                ])
            ])
        else:
            geo_string = "\n".join([
                geo_string,
                ", ".join(row[1])
            ])
    return(geo_string)


def read_last_month(data_dir):
    """
    Parameter
    ---------
    data_dir: str
        current month's data directory
    
    Returns
    -------
    last_month: DataFrame
        last month's data, if available
    """
    data_dir = dir_last_month(
        date(
            *eval(
                ",".join([
                    *os.path.basename(
                        data_dir
                    ).split(
                        '-'
                    ),
                    str(1)
                ])
            )
        )
    )
    last_month = os.path.join(
        data_dir,
        "monthly_numbers.csv"
    )
    return(
        pd.read_csv(
            last_month,
            index_col="Unnamed: 0",
            dtype={
                "CMI Total":int,
                "HBN":int,
                "FCP/INDI":int
            }
        ) if os.path.exists(
            last_month
        ) else pd.DataFrame(
            data={
                "CMI Total":[0,0],
                "HBN":[0,0],
                "FCP/INDI":[0,0]
            },
            columns=[
                "CMI Total",
                "HBN",
                "FCP/INDI"
            ],
            index=[
                "No. of Cities accessed data",
                "No. of Countries accessed data"
            ]
        )
    )

# Acquire reports
- [FCP](https://www.nitrc.org/project/stats/download_report.php?group_id=296)
- [INDI](https://www.nitrc.org/project/stats/download_report.php?group_id=404)
- HBN: LORIS & COINS
- Mindboggle

In [18]:
from datetime import timedelta

In [22]:
data_dir = dir_last_month(date.today() - timedelta(2))

In [24]:
last_month = read_last_month(data_dir)
last_month

,CMI Total,HBN,FCP/INDI
No. of Cities accessed data,2761,40,2756
No. of Countries accessed data,101,14,100


In [33]:
FCP = pd.read_csv(
    os.path.join(data_dir, "FCP.csv"),
    usecols=[
        "Date",
        "Package",
        "Release",
        "File",
        "IP Address"
    ],
    low_memory=False
)

INDI = pd.read_csv(
    os.path.join(data_dir, "INDI.csv"),
    usecols=[
        "Date",
        "Package",
        "Release",
        "File",
        "IP Address"
    ],
    low_memory=False
)

FCP_INDI = pd.concat([FCP, INDI])

HBN = pd.read_csv(
    os.path.join(data_dir, "HBN.csv")
)

In [34]:
FCP_INDI_locations_by_package = FCP_INDI[
    ["Package", "IP Address"]
].drop_duplicates(
    ["Package", "IP Address"]
).copy().reset_index(drop=True)

In [35]:
FCP_locations = FCP_INDI_locations_by_package["IP Address"].apply(
    lambda x: geo_from_ip(x)
).apply(pd.Series)

In [36]:
FCP_INDI_locations_by_package["City"] = FCP_locations[0]
FCP_INDI_locations_by_package["State or Province"] = FCP_locations[1]
FCP_INDI_locations_by_package["Country"] = FCP_locations[2]

In [37]:
FCP_INDI_locations_by_package = FCP_INDI_locations_by_package[
    ["Package", "City", "State or Province", "Country"]
].drop_duplicates(
    ["Package", "City", "State or Province", "Country"]
).copy()

In [38]:
FCP_INDI_locations = FCP_INDI_locations_by_package[
    ["City", "State or Province", "Country"]
].drop_duplicates(
    ["City", "State or Province", "Country"]
).copy().sort_values(
    ["Country", "City", "State or Province"]
)

In [39]:
HBN_locations = HBN["Location"].apply(lambda x: locsplit(x)).apply(pd.Series)
HBN_locations[["City", "State or Province", "Country"]] = HBN_locations[[0, 1, 2]]
HBN_locations = HBN_locations.drop([0, 1, 2], axis=1).copy()

In [40]:
HBN_locations = HBN_locations.drop_duplicates(
    ["City", "State or Province", "Country"]
).copy().sort_values(
    ["Country", "City", "State or Province"]
)

In [41]:
geo_count(FCP_INDI_locations, "city")

2984

In [42]:
geo_count(FCP_INDI_locations, "country")

101

In [43]:
geo_count(HBN_locations, "city")

43

In [44]:
geo_count(HBN_locations, "country")

14

In [45]:
all_locations = pd.concat([
    geo_frame(FCP_INDI_locations, "city"),
    geo_frame(HBN_locations, "city")
]).sort_values(
    ["Country", "City", "State or Province"]
).drop_duplicates(
    ["Country", "City", "State or Province"]
)

In [46]:
geo_count(all_locations, "city")

2993

In [47]:
geo_count(all_locations, "country")

102

In [48]:
print(print_geo_frame(HBN_locations))


Melbourne, Victoria, Australia
Leuven, Belgium
Porto Alegre, Brazil
London, Ontario, Canada
Toronto, Ontario, Canada
Beijing, China
Chengdu, Sichuan, China
Guangzhou, China
Jinzhong, China
Lanzhou, China
Nanjing, China
Shanghai, China
Shenzhen, China
Aglantzia, Cyprus
Gif Sur Yvette, France
Paris, France
Jülich, Germany
München, Germany
Bangalore, India,
Beer-Sheva, Israel
Doha, Qatar
Zurich, Switzerland
London, United Kingdom
Ames, Iowa, United States
Ann Arbor, Michigan, United States
Atlanta, Georgia, United States
Austin, Texas, United States
Baltimore, Maryland, United States
Bethesda, Maryland, United States
Bronx, New York, United States
Cambridge, Massachusetts, United States
College Station, Texas, United States
Columbia, South Carolina, United States
Houston, Texas, United States
Louisville, Kentucky, United States
New Haven, Connecticut, United States
New York, New York, United States
Orangeburg, New York, United States
Palo Alto, California, United States
Philadelphia, Pen

In [49]:
print(print_geo_frame(all_locations))


Algeria
Beni Mester, Tlemcen, Algeria
Constantine, Constantine, Algeria
Jijel, Jijel, Algeria
Oran, Oran, Algeria
Tindouf, Tindouf, Algeria
Tizi Rached, Tizi Ouzou, Algeria
Andorra la Vella, Andorra la Vella, Andorra
Argentina
Bahía Blanca, Buenos Aires, Argentina
Bariloche, Rio Negro, Argentina
Buenos Aires, Buenos Aires, Argentina
Burzaco, Buenos Aires, Argentina
Candelaria, Misiones, Argentina
Caseros, Buenos Aires, Argentina
Córdoba, Cordoba, Argentina
Ezpeleta, Buenos Aires, Argentina
Flores, Buenos Aires F.D., Argentina
General San Martin, Buenos Aires, Argentina
Godoy Cruz, Mendoza, Argentina
Hurlingham, Buenos Aires, Argentina
Ingeniero Beaugey, Buenos Aires, Argentina
La Plata, Buenos Aires, Argentina
Lanus, Buenos Aires, Argentina
Mar del Plata, Buenos Aires, Argentina
Moron, Buenos Aires, Argentina
Olivos, Buenos Aires, Argentina
Paraná, Entre Rios, Argentina
Rosario, Santa Fe, Argentina
Salta, Salta, Argentina
San Isidro, Buenos Aires, Argentina
Santo Tome, Santa Fe, Argen

In [50]:
last_month

,CMI Total,HBN,FCP/INDI
No. of Cities accessed data,2761,40,2756
No. of Countries accessed data,101,14,100


In [51]:
this_month = pd.DataFrame(
    data=dict(ChainMap(*[
        max_now_then(
            *a,
            last_month
        ) for a in [
            ("CMI Total", all_locations),
            ("HBN", HBN_locations),
            ("FCP/INDI",FCP_INDI_locations)
        ]
    ])),
    columns=[
        "CMI Total",
        "HBN",
        "FCP/INDI"
    ],
    index=[
        "No. of Cities accessed data",
        "No. of Countries accessed data"
    ]
)

In [52]:
this_month

,CMI Total,HBN,FCP/INDI
No. of Cities accessed data,2993,43,2984
No. of Countries accessed data,102,14,101


In [53]:
this_month.to_csv(
    os.path.join(
        data_dir,
        "monthly_numbers.csv"
    )
)